In [3]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357277 sha256=eec711b02b43869da47d1b82e9c4c32afb387f2c398974124c1825735a347f61
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [4]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import KFold
import pandas as pd

reader = Reader()
ratings = pd.read_csv('ratings.csv')
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [5]:
kf = KFold(n_splits=5)

In [9]:
from surprise import accuracy
svd = SVD()
for trainset, testset in kf.split(data):
    algo = svd
    algo.fit(trainset)
    predictions = algo.test(testset)
    rmse = accuracy.rmse(predictions, verbose=True)
    mae = accuracy.mae(predictions, verbose=True)

RMSE: 1.7848
MAE:  1.4554
RMSE: 1.7836
MAE:  1.4558
RMSE: nan
MAE:  nan
RMSE: 1.7842
MAE:  1.4514
RMSE: 1.7872
MAE:  1.4544


In [11]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [13]:
prediction = svd.predict(1, 30)
estimated_rating = prediction.est
print(f"Estimated rating for user 1 on movie 30: {estimated_rating}")

Estimated rating for user 1 on movie 30: 5


In [14]:
# now we have our colaborative filtering method ready lets work on contentBased one also

In [19]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

movies_metadata = pd. read_csv('movies_metadata.csv', on_bad_lines='skip', engine='python')
from ast import literal_eval
movies_metadata['genres'] = movies_metadata['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [20]:
vote_counts = movies_metadata[movies_metadata['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = movies_metadata[movies_metadata['vote_average'].notnull()]['vote_average'].astype('int')

In [21]:
movies_metadata['year'] = pd.to_datetime(movies_metadata['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [27]:
keywords = pd.read_csv('keywords.csv', on_bad_lines='skip', engine='python')


ratings = pd.read_csv('ratings.csv', on_bad_lines='skip', engine='python')

In [41]:
# credits = pd.read_csv('credits.csv', on_bad_lines='skip', engine='python')
links = pd.read_csv('links.csv', on_bad_lines='skip', engine='python')

In [42]:
links.columns

Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')

In [43]:
links = links[links['tmdbId'].notnull()]['tmdbId'].astype('int')
# movies_metadata = movies_metadata.drop([19730, 29503, 35587])
movies_metadata['id'] = movies_metadata['id'].astype('int')

In [44]:
movies_metadata = movies_metadata[movies_metadata['id'].isin(links)]

In [45]:
movies_metadata['title']

,title
0,Toy Story
1,Jumanji
2,Grumpier Old Men
3,Waiting to Exhale
4,Father of the Bride Part II
...,...
3807,The Giant Gila Monster
3808,The Invisible Man
3809,The Killer Shrews
3810,Kronos


In [46]:
movies_metadata['tagline'] = movies_metadata['tagline'].fillna('')
movies_metadata['description'] = movies_metadata['overview'] + movies_metadata['tagline']
movies_metadata['description'] = movies_metadata['description'].fillna('')

In [47]:
credits = pd.read_csv('credits.csv', on_bad_lines='skip', engine='python')

In [48]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
movies_metadata['id'] = movies_metadata['id'].astype('int')

movies_metadata = movies_metadata.merge(credits, on='id')
movies_metadata = movies_metadata.merge(keywords, on='id')
movies_metadata = movies_metadata[movies_metadata['id'].isin(links)]


In [49]:
movies_metadata.shape

(156, 29)

In [50]:
movies_metadata['cast'] = movies_metadata['cast'].apply(literal_eval)
movies_metadata['crew'] = movies_metadata['crew'].apply(literal_eval)
movies_metadata['keywords'] = movies_metadata['keywords'].apply(literal_eval)
movies_metadata['cast_size'] = movies_metadata['cast'].apply(lambda x: len(x))
movies_metadata['crew_size'] = movies_metadata['crew'].apply(lambda x: len(x))

In [51]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

movies_metadata['director'] = movies_metadata['crew'].apply(get_director)
movies_metadata['cast'] = movies_metadata['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
movies_metadata['cast'] = movies_metadata['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

movies_metadata['keywords'] = movies_metadata['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
movies_metadata['cast'] = movies_metadata['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

movies_metadata['director'] = movies_metadata['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
movies_metadata['director'] = movies_metadata['director'].apply(lambda x: [x,x,x])
s = movies_metadata.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

,count
keyword,
woman director,14
independent film,14
murder,7
violence,6
based on novel,6


In [52]:
s = s[s > 1]

In [53]:
movies_metadata['director']

,director
0,"[johnlasseter, johnlasseter, johnlasseter]"
1,"[joejohnston, joejohnston, joejohnston]"
2,"[howarddeutch, howarddeutch, howarddeutch]"
3,"[forestwhitaker, forestwhitaker, forestwhitaker]"
4,"[charlesshyer, charlesshyer, charlesshyer]"
...,...
151,"[luisbuñuel, luisbuñuel, luisbuñuel]"
152,"[johnboorman, johnboorman, johnboorman]"
153,"[paulauster, paulauster, paulauster]"
154,"[michaelmoore, michaelmoore, michaelmoore]"


In [54]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

movies_metadata['keywords'] = movies_metadata['keywords'].apply(filter_keywords)
movies_metadata['keywords'] = movies_metadata['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
movies_metadata['keywords'] = movies_metadata['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [55]:
movies_metadata['cast'] = movies_metadata['cast'].apply(lambda x : [str(i) for i in x])
movies_metadata['keywords'] = movies_metadata['keywords'].apply(lambda x : [str(i) for i in x])
movies_metadata['director'] = movies_metadata['director'].apply(lambda x : [str(i) for i in x])
movies_metadata['genres'] = movies_metadata['genres'].apply(lambda x : [str(i) for i in x])
movies_metadata['total'] = movies_metadata['keywords'] + movies_metadata['cast'] + movies_metadata['director'] + movies_metadata['genres']
movies_metadata['total'] = movies_metadata['total'].apply(lambda x: ' '.join(x))

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2), stop_words='english')
tfidf_matrix = tf.fit_transform(movies_metadata['description'])

In [58]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

movies_metadata = movies_metadata.reset_index()
titles = movies_metadata['title']
indices = pd.Series(movies_metadata.index, index=movies_metadata['title'])

In [59]:
tfidf_matrix.shape

(156, 7686)

In [60]:
movies_metadata = movies_metadata.reset_index()
titles = movies_metadata['title']
indices = pd.Series(movies_metadata.index, index=movies_metadata['title'])

def get_similar_movies(title):
    idx = indices[title]
    dot_product = list(enumerate(cosine_sim[idx]))
    dot_product = sorted(dot_product, key=lambda x: x[1], reverse=True)
    movie_indices = [i[0] for i in dot_product]
    return titles.iloc[movie_indices]

In [62]:
movies_metadata['title']

,title
0,Toy Story
1,Jumanji
2,Grumpier Old Men
3,Waiting to Exhale
4,Father of the Bride Part II
...,...
151,Belle de Jour
152,Beyond Rangoon
153,Blue in the Face
154,Canadian Bacon


In [65]:
C = vote_averages.mean()
m = vote_counts.quantile(0.5)
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [64]:
def final_recommendations(title, percentile=0.50):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=(lambda x: x[1]), reverse=True)

    movie_indices = [i[0] for i in sim_scores]

    movies = movies_metadata.iloc[movie_indices][['title', 'vote_count', 'vote_average']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)

    temp = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    temp['vote_count'] = temp['vote_count'].astype('int')
    temp['vote_average'] = temp['vote_average'].astype('int')
    temp['wr'] = temp.apply(weighted_rating, axis=1)
    temp = temp.sort_values('wr', ascending=False).head(10)
    return temp

In [66]:
final_recommendations('Toy Story', 0.5)

,title,vote_count,vote_average,wr
46,Se7en,5915,8,7.978838
49,The Usual Suspects,3334,8,7.962717
109,Taxi Driver,2632,8,7.952972
0,Toy Story,5415,7,6.986777
108,Braveheart,3404,7,6.979087
31,Twelve Monkeys,2470,7,6.971349
5,Heat,1886,7,6.962724
147,Apollo 13,1637,7,6.957235
15,Casino,1343,7,6.948235
95,La Haine,695,7,6.903449


In [75]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [76]:
id_map = pd.read_csv('links.csv', on_bad_lines='skip', engine='python')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(movies_metadata[['title', 'id']], on='id').set_index('title')

In [79]:
indices_map = id_map.set_index('id')

In [80]:
def combined(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']

    movie_id = id_map.loc[title]['movieId']

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = movies_metadata.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [81]:
combined(1, 'Taxi Driver')

,title,vote_count,vote_average,year,id,est
98,City Hall,67,6.0,1996,11062,5
69,From Dusk Till Dawn,1644,6.9,1996,755,5
30,Dangerous Minds,249,6.4,1995,9909,5
138,Up Close & Personal,51,5.9,1996,9302,5
48,When Night Is Falling,10,5.9,1995,8391,5
110,Rumble in the Bronx,245,6.5,1995,33542,5
18,Ace Ventura: When Nature Calls,1128,6.1,1995,9273,5
144,The Basketball Diaries,472,7.2,1995,10474,5
17,Four Rooms,539,6.5,1995,5,5
150,Batman Forever,1529,5.2,1995,414,5
